In [1]:
import pandas as pd
data = pd.read_csv("/Users/ethanschultz/Downloads/AmesHousing.csv")
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,...,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,...,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,...,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
X = data.drop("SalePrice", axis = 1)
y = data["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:

ct = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [5]:
lr_fitted = lr_pipeline.fit(X_train, y_train)
train_pred = lr_fitted.predict(X_train)
test_pred = lr_fitted.predict(X_test)
test_mse = mean_squared_error(y_test, test_pred)
test_r2 = r2_score(y_test, test_pred)
intercept = lr_fitted.named_steps['linear_regression'].intercept_
coefficient = lr_fitted.named_steps['linear_regression'].coef_


In [6]:
print("Test MSE:", test_mse)
print("Test R^2:", test_r2)
print("intercept:", intercept)
print("coefficients:", coefficient)


Test MSE: 3141094417.503056
Test R^2: 0.5280071012924208
intercept: 182305.74419663177
coefficients: [ 68868.79172694 -16303.53573005]
Test MSE: 3141094417.503056
Test R^2: 0.5280071012924208
intercept: 182305.74419663177
coefficients: [ 68868.79172694 -16303.53573005]


Model 2

In [7]:

ct1 = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),    
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])

  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct1),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd']),
                                                 ('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type'])])),
                ('linear_regression', LinearRegression())])

In [8]:
lr_fitted = lr_pipeline.fit(X_train, y_train)
test_pred = lr_fitted.predict(X_test)

In [9]:

test_mse = mean_squared_error(y_test, test_pred)
test_r2 = r2_score(y_test, test_pred)
intercept2 = lr_fitted.named_steps['linear_regression'].intercept_
coefficient2 = lr_fitted.named_steps['linear_regression'].coef_

In [10]:
print("Test MSE:", test_mse)
print("Test R^2:", test_r2)
print("intercept:", intercept2)
print("coefficients:", coefficient2)

Test MSE: 2895371953.919885
Test R^2: 0.5649303014413622
intercept: 163217.56966190095
coefficients: [ 63856.64219832  -9376.1959482   21296.7532576  -27732.92949287
 -32408.36399729  -2164.85798743  41009.39821998]
Test MSE: 2895371953.919885
Test R^2: 0.5649303014413622
intercept: 163217.56966190095
coefficients: [ 63856.64219832  -9376.1959482   21296.7532576  -27732.92949287
 -32408.36399729  -2164.85798743  41009.39821998]


In [11]:
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
  ("standardize", StandardScaler(), ["Gr Liv Area"])
  ],
  remainder = "passthrough"
).set_output(transform = "pandas")

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["standardize__GR Liv Area", "dummify__Bldg Type_1Fam"]),
  ],
  remainder = "drop"
).set_output(transform = "pandas")

X_train_dummified = ct_dummies.fit_transform(X_train)
X_train_dummified

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,standardize__Gr Liv Area,remainder__Order,remainder__PID,remainder__MS SubClass,remainder__MS Zoning,remainder__Lot Frontage,remainder__Lot Area,remainder__Street,remainder__Alley,remainder__Lot Shape,remainder__Land Contour,remainder__Utilities,remainder__Lot Config,remainder__Land Slope,remainder__Neighborhood,remainder__Condition 1,remainder__Condition 2,remainder__House Style,remainder__Overall Qual,remainder__Overall Cond,remainder__Year Built,remainder__Year Remod/Add,remainder__Roof Style,remainder__Roof Matl,remainder__Exterior 1st,remainder__Exterior 2nd,remainder__Mas Vnr Type,remainder__Mas Vnr Area,remainder__Exter Qual,remainder__Exter Cond,remainder__Foundation,remainder__Bsmt Qual,remainder__Bsmt Cond,remainder__Bsmt Exposure,remainder__BsmtFin Type 1,...,remainder__Heating,remainder__Heating QC,remainder__Central Air,remainder__Electrical,remainder__1st Flr SF,remainder__2nd Flr SF,remainder__Low Qual Fin SF,remainder__Bsmt Full Bath,remainder__Bsmt Half Bath,remainder__Full Bath,remainder__Half Bath,remainder__Bedroom AbvGr,remainder__Kitchen AbvGr,remainder__Kitchen Qual,remainder__TotRms AbvGrd,remainder__Functional,remainder__Fireplaces,remainder__Fireplace Qu,remainder__Garage Type,remainder__Garage Yr Blt,remainder__Garage Finish,remainder__Garage Cars,remainder__Garage Area,remainder__Garage Qual,remainder__Garage Cond,remainder__Paved Drive,remainder__Wood Deck SF,remainder__Open Porch SF,remainder__Enclosed Porch,remainder__3Ssn Porch,remainder__Screen Porch,remainder__Pool Area,remainder__Pool QC,remainder__Fence,remainder__Misc Feature,remainder__Misc Val,remainder__Mo Sold,remainder__Yr Sold,remainder__Sale Type,remainder__Sale Condition
552,1.0,0.0,0.0,0.0,0.0,-1.304052,553,531477020,20,RH,60.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Feedr,Norm,1Story,4,4,1950,1950,Gable,CompShg,Wd Sdng,AsbShng,NaN,0.0,Fa,Fa,CBlock,TA,Fa,No,Unf,...,GasA,Gd,Y,SBrkr,841,0,0,0.0,0.0,1,0,2,1,TA,4,Typ,0,NaN,CarPort,1950.0,Unf,1.0,294.0,TA,TA,N,250,0,24,0,0,0,NaN,NaN,NaN,0,9,2009,WD,Normal
2101,1.0,0.0,0.0,0.0,0.0,-0.373963,2102,906380010,20,RL,103.0,11175,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,Norm,Norm,1Story,7,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,Gd,TA,PConc,Gd,TA,Av,Unf,...,GasA,Ex,Y,SBrkr,1316,0,0,0.0,0.0,2,0,3,1,Gd,6,Typ,1,Gd,Attchd,2007.0,Fin,2.0,440.0,TA,TA,Y,0,20,0,0,0,0,NaN,NaN,NaN,0,10,2007,New,Partial
931,1.0,0.0,0.0,0.0,0.0,0.781305,932,909428240,20,RL,100.0,17500,Pave,NaN,Reg,Bnk,AllPub,Corner,Mod,Crawfor,PosA,Norm,1Story,6,5,1954,1954,Gable,CompShg,HdBoard,HdBoard,Stone,420.0,TA,TA,PConc,TA,TA,Av,LwQ,...,GasA,Ex,Y,SBrkr,1906,0,0,1.0,0.0,1,1,3,1,TA,6,Typ,2,Gd,Basment,1954.0,Unf,2.0,576.0,TA,TA,Y,0,201,0,0,0,0,NaN,NaN,NaN,0,1,2009,WD,Normal
2186,0.0,0.0,1.0,0.0,0.0,-0.610891,2187,908225310,90,RL,52.0,8741,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Story,5,6,1946,1950,Gable,CompShg,Plywood,Plywood,NaN,0.0,TA,TA,CBlock,TA,TA,No,Unf,...,GasA,TA,N,SBrkr,1195,0,0,0.0,0.0,2,0,4,2,TA,8,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,N,118,0,0,0,0,0,NaN,GdWo,NaN,0,7,2007,WD,Abnorml
1674,0.0,0.0,0.0,0.0,1.0,-0.332843,1675,527427200,120,RH,26.0,10943,Pave,NaN,IR2,Lvl,AllPub,FR2,Gtl,NAmes,Norm,Norm,1Story,6,5,1997,1998,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,...,GasA,Ex,Y,SBrkr,1337,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,1,TA,Attchd,1997.0,Fin,2.0,522.0,TA,TA,Y,0,68,0,0,0,0,NaN,NaN,NaN,0,10,2007,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,1.0,0.0,0.0,0.0,0.0,-0.890897,772,905103070,20,RL,75.0,11625,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Story,5,6,19